In [10]:
from catboost import CatBoostClassifier
import pandas as pd
import json
from pandas import json_normalize
from tqdm import tqdm_notebook as tqdm
import numpy as np
import ast

use_cols = ['hitsPM', 'accentedHitsPM', 'takedownsPM', 'accentedHitsDistancePM',
       'accentedHitsClinchPM', 'accentedHitsParterPM', 'hitsSuccessfulPM',
       'accentedHitsSuccessfulPM', 'takedownsSuccessfulPM',
       'accentedHitsDistanceSuccessfulPM', 'accentedHitsClinchSuccessfulPM',
       'accentedHitsParterSuccessfulPM', 'hitsSuccessful_percent',
       'accentedHitsSuccessful_percent', 'accentedHits_percent',
       'takedownSuccessful_percent',
       'accentedHitsPositionDistanceSuccessful_prcent',
       'accentedHitsPositionClinchSuccessful_percent',
       'accentedHitsPositionParterSuccessful_percent', 'takedowns_to_hits',
       'HitsPositionDistance_to_hits', 'HitsPositionClinch_to_hits',
       'HitsPositionParter_to_hits', 'odd_diff',
       'fighter1_fightsAmount', 'fighter2_fightsAmount']#, 'odd1', 'odd2']


In [11]:
import catboost
print(catboost.__version__)

0.26.1


In [12]:
train_df_combined = pd.read_pickle('data/train_df_combined.pkl')
valid_df_combined = pd.read_pickle('data/valid_df_combined.pkl')
test_df_combined = pd.read_pickle('data/valid_df_combined.pkl')
train_df_combined.shape, valid_df_combined.shape, test_df_combined.shape

((2622, 30), (501, 30), (501, 30))

In [13]:
X_train = train_df_combined[use_cols]
y_train = train_df_combined['winner'].astype(int)

X_val = valid_df_combined[use_cols]
y_val = valid_df_combined['winner'].astype(int)

# clf = CatBoostClassifier(custom_metric=['Accuracy'], max_depth=3, iterations=100)
clf = CatBoostClassifier(iterations=1000, 
                           depth=4,
                           learning_rate=0.01,
                           loss_function='Logloss',
                           colsample_bylevel=1,
                           subsample=0.5,
#                                    bagging_temperature=bagging_temperature,
                           random_strength=1,
                           l2_leaf_reg=30,
                           eval_metric='Accuracy',
#                            boosting_type='Plain', # 'Plain'
                           bootstrap_type='Bernoulli',# 'Bernoulli' Bayesian
                           max_ctr_complexity=2,
                           task_type='GPU',
                          )

clf.fit(X_train, y_train, plot=True, eval_set=(X_val, y_val), verbose=False)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [14]:
y_pred = clf.predict(test_df_combined[clf.feature_names_])
test_df_combined['y_pred'] = y_pred
test_df_combined

,hitsPM,accentedHitsPM,takedownsPM,accentedHitsDistancePM,accentedHitsClinchPM,accentedHitsParterPM,hitsSuccessfulPM,accentedHitsSuccessfulPM,takedownsSuccessfulPM,accentedHitsDistanceSuccessfulPM,...,HitsPositionClinch_to_hits,HitsPositionParter_to_hits,eventDate.date,winner,odd_diff,fighter1_fightsAmount,fighter2_fightsAmount,odd1,odd2,y_pred
2622,-25.89794,-19.612038,-0.024196,1.899624,0.768617,-22.280279,-16.028251,-11.261337,-0.047324,0.431535,...,0.023541,-0.200478,2019-01-20 02:00:00,False,4.28,2,3,5.48,1.20,0
2623,8.413548,8.093302,-1.621034,5.983601,-0.272975,2.382676,4.024911,3.855419,0.0,2.21182,...,-0.020833,0.378571,2019-01-20 02:00:00,True,-2.35,2,3,1.40,3.75,1
2624,-7.590965,1.043771,-0.55089,4.832289,-4.653893,0.865375,-10.658257,-3.237961,1.297714,1.852607,...,-0.042848,-0.062206,2019-01-20 02:00:00,False,-0.26,1,1,1.89,2.15,1
2625,-57.506089,-49.296309,-0.098447,-50.491969,-2.983843,4.179503,-25.856508,-18.955591,0.50037,-19.852446,...,0.098823,0.066837,2019-01-20 02:00:00,True,2.41,2,1,3.69,1.28,1
2626,-13.334967,-13.409557,0.355405,-3.249364,0.375843,-10.536036,-13.812913,-13.756783,0.135135,-2.909212,...,0.033703,-0.164385,2019-01-20 02:00:00,True,3.81,2,4,5.01,1.20,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3118,-2.840035,-7.628505,-0.259648,-13.296777,-0.097671,5.765943,4.640153,0.32964,-0.057143,-4.330109,...,-0.037361,0.222779,2019-12-21 00:00:00,True,-0.05,2,1,1.89,1.94,1
3119,-11.663098,-10.051576,-0.288376,-6.577657,-2.030127,-1.443792,-8.69991,-7.243997,-0.103094,-4.383578,...,0.009193,-0.114519,2019-12-21 00:00:00,False,1.45,3,3,2.88,1.43,0
3120,-33.480823,-35.842107,-0.252101,-34.181974,-1.258317,-0.401816,-11.349771,-13.393457,0.0,-11.685426,...,0.066178,-0.310247,2019-12-21 00:00:00,False,-0.14,2,2,1.85,1.99,1
3121,3.739086,4.619619,0.30794,3.676759,0.001156,0.941704,5.649491,6.230024,0.30794,4.703043,...,0.001948,0.178571,2019-12-21 00:00:00,True,1.07,1,2,2.58,1.51,1


,hitsPM,accentedHitsPM,takedownsPM,accentedHitsDistancePM,accentedHitsClinchPM,accentedHitsParterPM,hitsSuccessfulPM,accentedHitsSuccessfulPM,takedownsSuccessfulPM,accentedHitsDistanceSuccessfulPM,...,HitsPositionClinch_to_hits,HitsPositionParter_to_hits,eventDate.date,winner,odd_diff,fighter1_fightsAmount,fighter2_fightsAmount,odd1,odd2,y_pred
2622,-25.89794,-19.612038,-0.024196,1.899624,0.768617,-22.280279,-16.028251,-11.261337,-0.047324,0.431535,...,0.023541,-0.200478,2019-01-20 02:00:00,False,4.28,2,3,5.48,1.20,0
2623,8.413548,8.093302,-1.621034,5.983601,-0.272975,2.382676,4.024911,3.855419,0.0,2.21182,...,-0.020833,0.378571,2019-01-20 02:00:00,True,-2.35,2,3,1.40,3.75,1
2624,-7.590965,1.043771,-0.55089,4.832289,-4.653893,0.865375,-10.658257,-3.237961,1.297714,1.852607,...,-0.042848,-0.062206,2019-01-20 02:00:00,False,-0.26,1,1,1.89,2.15,1
2625,-57.506089,-49.296309,-0.098447,-50.491969,-2.983843,4.179503,-25.856508,-18.955591,0.50037,-19.852446,...,0.098823,0.066837,2019-01-20 02:00:00,True,2.41,2,1,3.69,1.28,1
2626,-13.334967,-13.409557,0.355405,-3.249364,0.375843,-10.536036,-13.812913,-13.756783,0.135135,-2.909212,...,0.033703,-0.164385,2019-01-20 02:00:00,True,3.81,2,4,5.01,1.20,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3118,-2.840035,-7.628505,-0.259648,-13.296777,-0.097671,5.765943,4.640153,0.32964,-0.057143,-4.330109,...,-0.037361,0.222779,2019-12-21 00:00:00,True,-0.05,2,1,1.89,1.94,1
3119,-11.663098,-10.051576,-0.288376,-6.577657,-2.030127,-1.443792,-8.69991,-7.243997,-0.103094,-4.383578,...,0.009193,-0.114519,2019-12-21 00:00:00,False,1.45,3,3,2.88,1.43,0
3120,-33.480823,-35.842107,-0.252101,-34.181974,-1.258317,-0.401816,-11.349771,-13.393457,0.0,-11.685426,...,0.066178,-0.310247,2019-12-21 00:00:00,False,-0.14,2,2,1.85,1.99,1
3121,3.739086,4.619619,0.30794,3.676759,0.001156,0.941704,5.649491,6.230024,0.30794,4.703043,...,0.001948,0.178571,2019-12-21 00:00:00,True,1.07,1,2,2.58,1.51,1


In [15]:
print('Точность модели')
(test_df_combined['y_pred'] == test_df_combined['winner']).mean()

Точность модели


0.6387225548902196

Точность модели


0.6387225548902196

In [16]:
print('Точность букмейкера')
test_df_combined['bookmaker_bet'] = (test_df_combined['odd1'] < test_df_combined['odd2']).astype(int)
(test_df_combined['bookmaker_bet'] == test_df_combined['winner']).mean()

Точность букмейкера


0.6187624750499002

Точность букмейкера


0.6187624750499002

In [17]:
def prof_acc(y_true, y_pred, odds1, odds2):
    odds = list(zip(odds1, odds2))
    avg_odd = []

    for num, boo in enumerate(y_true == y_pred):
        if boo == True:
            pred = y_true[num]
            if pred == 0: pred = 1
            else: pred = 0
            res = odds[num][pred]
            avg_odd.append(res) 
        if boo == False:
            pred = y_true[num]
            if pred == 0: pred = 1
            else: pred = 0

            res = odds[num][pred]
            avg_odd.append(-1)
    print('Average ROI:', (np.mean(avg_odd)-1)*100, '%')
    return (np.mean(avg_odd)-1)*100

prof_acc(test_df_combined['winner'].values, test_df_combined['y_pred'].values, test_df_combined['odd1'].values, test_df_combined['odd2'].values)

Average ROI: -21.125748502994014 %


-21.125748502994014

Average ROI: -21.125748502994014 %


-21.125748502994014

# Простейшая стратегия ставок

In [18]:
bet = 100 # рублей
bank = 0 
for ind in tqdm(test_df_combined.index):
    winner, y_pred, odd1, odd2 = test_df_combined.loc[ind, ['winner', 'y_pred', 'odd1', 'odd2']]
    
    if winner == y_pred:
        if winner:
            win_odd = odd1
        else: win_odd = odd2
        
        bank += bet*win_odd - bet
    
    else:
        bank -= bet
bank

/tmp/ipykernel_25593/1184950811.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for ind in tqdm(test_df_combined.index):


  0%|          | 0/501 [00:00<?, ?it/s]

7516.0

/tmp/ipykernel_25593/1184950811.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for ind in tqdm(test_df_combined.index):


  0%|          | 0/501 [00:00<?, ?it/s]

7516.0